In [ ]:
import os
import datetime
import pandas as pd
import numpy as np
import requests

from IPython.display import display, HTML

from settings import *

In [ ]:
if 'ALPHAVANTAGE_KEY' not in os.environ:
    raise ValueError('Need to define the API_KEY for the quotes')
    
ALPHAVANTAGE_KEY = os.environ['ALPHAVANTAGE_KEY']
URL='https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=full&apikey={}'.format(ASSET_TO_CALCULATE,ALPHAVANTAGE_KEY)

response = requests.get(URL)
if response.status_code != 200:
    raise ValueError('Error in getting the Data')
    
response = response.json()

data_to_process = response['Time Series (Daily)']
days = list(data_to_process.keys())

In [ ]:
data_for_pandas = []
for current_day in days:
    current_day_date = datetime.datetime.strptime(current_day, '%Y-%m-%d')
    if current_day_date < DATE_MINIMAL:
        continue
    data_for_pandas.append(
        {
            'Date': current_day.replace('-',''),
            'Price': float(data_to_process[current_day]['4. close']),
            'Open':  float(data_to_process[current_day]['1. open']),
            'High':  float(data_to_process[current_day]['2. high']),
            'Low':  float(data_to_process[current_day]['3. low']),
            'Vol':  float(data_to_process[current_day]['6. volume'])
        }
    )
data_for_pandas = sorted(data_for_pandas, key=lambda x: x['Date'], reverse=False)

In [ ]:
df = pd.DataFrame(data_for_pandas)

In [ ]:
output_csv_file='./input/{}_Daily.csv'.format(ASSET_TO_CALCULATE)
df.to_csv(output_csv_file)

In [ ]:
output_excel_file='./input/{}_Daily.xlsx'.format(ASSET_TO_CALCULATE)
df.to_excel(output_excel_file)